In [1]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate();

  Activating project at `~/Documents/eth_courses/notebooks/dynamics/julia/demos/quad_2d`


In [12]:
using GLMakie
using Colors
using GeometryBasics
using Rotations
using FileIO

include("gui_utilities.jl")

GLMakie.activate!(inline=false)

In [13]:
Pkg.add("FileIO")

   Resolving package versions...
  No Changes to `~/Documents/eth_courses/notebooks/dynamics/julia/demos/quad_2d/Project.toml`
  No Changes to `~/Documents/eth_courses/notebooks/dynamics/julia/demos/quad_2d/Manifest.toml`


### Load Mesh

In [14]:
crazyflie_stl = load(assetpath(pwd() * "/assets/cf2_assembly.obj"));

### Create figure 

In [62]:
set_theme!(
#     font = "Arial", # inherited by layoutables if not overridden
    fontsize = 25, # inherited by layoutables if not overridden
)

f = Figure(resolution = (2560, 1920))



display(f)

GLMakie.Screen(...)

In [75]:
# clear figure 
empty!(f)

vis_params = (;aspect=1.0)
axis_params = (x_low=-2, x_high=2, y_low=-2, y_high=2, z_low=-2, z_high=2, aspect_x=1, aspect_y=1, aspect_z=1, label_size=40)
model_params = (;scale_x=10.5, scale_y=10.5, scale_z=10.5, initial_translation = Vec3f(0.0, 0.0, 0.0))

# Top level
g_left = f[1, 1] = GridLayout(alignmode=Outside(50))
g_right = f[1, 2] = GridLayout(alignmode=Outside(30))

# # red box for control panel 
# Box(f[1, 2], color = (:red, 0.2), strokewidth = 0)

# force 3d visualizer to have an aspect ratio of 1
colsize!(f.layout, 1, Aspect(1, vis_params.aspect))

# # Column size adjust
colsize!(g_left, 1,  Auto(1))
colsize!(g_right, 1,  Auto(1))

# 3d axis for airplane visualization
ax1 = Axis3(g_left[1, 1],
    title = "",
    limits =  (axis_params.x_low,axis_params.x_high, axis_params.y_low,axis_params.y_high, axis_params.z_low,axis_params.z_high),
    aspect = (axis_params.aspect_x, axis_params.aspect_y, axis_params.aspect_z),
    xlabel="x axis", xlabelsize=axis_params.label_size,
    ylabel="y axis", ylabelsize=axis_params.label_size,
    zlabel="z axis", zlabelsize=axis_params.label_size,
    halign =:left
)

m = mesh!(ax1, crazyflie_stl , color=:red)

scale!(m, model_params.scale_x, model_params.scale_y, model_params.scale_z)

# center model at the origin
translate!(m, model_params.initial_translation)

# orient along positive x axis
rotate!(m, Vec3f(0, 1, 0), 0) # 0.5 rad around the y axis


# Right Grids - plots
g_right_plots = GridLayout()
g_right_widgets = GridLayout()

plot_1 = Axis(f)
plot_2 = Axis(f)

g_right_plots[2,1] = plot_1
g_right_plots[3,1] = plot_2
g_right[1, 1] = g_right_plots

# Right Grids - widgets

# slider grid for setting attitude
attitude_slider = SliderGrid(
    f,
    (label = "Roll", range = -pi:0.01:pi, format = "{:.2f} rad", startvalue = 0.0, linewidth = 25.0),
    (label = "Pitch", range = -pi:0.01:pi, format = "{:.2f} rad", startvalue = 0.0, linewidth = 25.0),
    (label = "Yaw", range = -pi:0.01:pi, format = "{:.2f} rad", startvalue = 0.0, linewidth = 25.0),
    width = Auto(),
    tellheight = false,
    halign =:left
)

# attitude reset button
attitude_reset_btn = Button(f, label = "Reset Attitude", tellwidth=false)

g_right_widgets[1,1] = attitude_slider
g_right_widgets[2,1] = attitude_reset_btn
g_right[2, 1] = g_right_widgets

# Observables
attitude_slider_obs = [s.value for s in attitude_slider.sliders]


# events

# Attitude button clicked
on(attitude_reset_btn.clicks) do clicks
    # reset attitude
    rotate!(m, Quaternion(1,0,0,0))  
    
    # reset the attitude control sliders
    set_close_to!(attitude_slider.sliders[1], attitude_slider.sliders[1].startvalue[])
    set_close_to!(attitude_slider.sliders[2], attitude_slider.sliders[1].startvalue[])
    set_close_to!(attitude_slider.sliders[3], attitude_slider.sliders[1].startvalue[])
end


bars = lift(attitude_slider_obs...) do angles...
    
    # create ZYX euler angle representation 
    R_euler = RotZYX(angles[1],angles[2],angles[3])
    
    # Convert the rotation to unit quaternion
    R_quat = QuatRotation(R_euler)
    
    R_quat_makie = Quaternion(R_quat.q.s, R_quat.q.v1,R_quat.q.v2,R_quat.q.v3)
    
    # apply rotation
    rotate!(m,  R_quat_makie)     
end

trim!(f.layout)